**Bokeh charting example for CPI, Core CPI, and Stick CPI**

In [1]:
import pandas as pd
from requests import get
from fake_useragent import UserAgent
from io import StringIO

keys = ["CPIAUCSL", "CPILFESL", "CORESTICKM159SFRBATL"]
ColNames = ["Date", "CPI", "Core", "Sticky"]
start, end = pd.Timestamp(2005, 1, 1), pd.Timestamp.today()

dfs = []
headers = {"User-Agent": UserAgent().chrome}
for i in range(len(keys)):
    url = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=" + keys[i]
    df = pd.read_csv(StringIO(get(url, headers=headers).text))

    df["DATE"] = pd.to_datetime(df["DATE"])
    df = df.loc[(df["DATE"] >= start) & (df["DATE"] <= end)]
    df = df.reset_index(drop=True)

    if df[keys[i]][0] >= 150:
        df[keys[i]] = df[keys[i]].pct_change(12, fill_method="ffill") * 100
    dfs.append(df)

df = pd.concat(dfs, axis=1).dropna().reset_index(drop=True)
df = df.loc[:, ~df.columns.duplicated()]
df.columns = ColNames
df


,Date,CPI,Core,Sticky
0,2006-01-01,4.018789,2.110553,2.451019
1,2006-02-01,3.638254,2.106319,2.499255
2,2006-03-01,3.417918,2.098951,2.581219
3,2006-04-01,3.613836,2.297702,2.680693
4,2006-05-01,3.977273,2.443890,2.783959
...,...,...,...,...
189,2021-10-01,6.236941,4.593958,3.042940
190,2021-11-01,6.828372,4.952634,3.205069
191,2021-12-01,7.096542,5.476621,3.467011
192,2022-01-01,7.525934,6.040783,3.982988


In [2]:
from bokeh.io import output_notebook
from FRED_Bokeh import *

output_notebook()

show(chartFRED(df))


Loading BokehJS ...